In [2]:
##import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import string
from sklearn.utils import shuffle
import tqdm
from keras.models import load_model
import emoji

Using TensorFlow backend.


In [3]:
data = pd.read_csv("../Highbrowsent/sentiment140/training.1600000.processed.noemoticon.csv",sep=',',header=None, encoding='ISO-8859-1')

In [4]:
data.head()

,0,1,2,3,4,5
0,0,1553733392,Sat Apr 18 15:03:53 PDT 2009,NO_QUERY,vanaessa,Benny's out.
1,0,1553733396,Sat Apr 18 15:03:54 PDT 2009,NO_QUERY,thespiderman86,I AM SO SAD BABE!!! FOB!!! I KNOW
2,0,1553733483,Sat Apr 18 15:03:52 PDT 2009,NO_QUERY,my_light,My garbage disposal is broken.
3,0,1553733516,Sat Apr 18 15:03:53 PDT 2009,NO_QUERY,BaByDoll79,Got my nails done..y did electric pink turn in...
4,0,1553733543,Sat Apr 18 15:03:53 PDT 2009,NO_QUERY,jpadamson,@rinygrin Me and Al were going to call you las...


In [4]:
X = data.iloc[:,5:6].values

In [5]:
Y = data.iloc[:,0:1].values

In [6]:
X.shape

(1600000, 1)

In [7]:
Y.shape

(1600000, 1)

In [8]:
#preprocessing lib
import re

In [ ]:
#preprocessing
corpus = []
for i in tqdm.tqdm(range(X.shape[0])):
    frame = re.sub('[^a-zA-Z]', ' ', X[i][0])
    frame = frame.lower()
    frame = frame.split()
    frame = ' '.join(frame)
    corpus.append(frame)

In [ ]:
X_new = np.asarray(corpus)

In [ ]:
X_new = np.reshape(X_new, (len(corpus),1))

In [ ]:
X_new.shape

In [ ]:
Y.shape

In [ ]:
from sklearn.utils import shuffle
X_new, Y = shuffle(X_new, Y)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X_new, Y, test_size=0.25, random_state=42)

In [ ]:
X_test, X_val, Y_test, Y_val = train_test_split(X_test, Y_test, test_size=0.5, random_state=42)

In [15]:
X_train.shape

(1200000, 1)

In [16]:
X_test.shape

(200000, 1)

In [17]:
X_val.shape

(200000, 1)

In [ ]:
Y_train = Y_train / 4
Y_test = Y_test / 4
Y_val = Y_val / 4

In [9]:
X_train = np.load('../input/processed/X_h_train.npy')

In [10]:
Y_train = np.load('../input/processed/Y_h_train.npy')

In [11]:
X_test = np.load('../input/processed/X_h_test.npy')

In [12]:
Y_test = np.load('../input/processed/Y_h_test.npy')

In [13]:
X_val = np.load('../input/processed/X_h_val.npy')

In [14]:
Y_val = np.load('../input/processed/Y_h_val.npy')

In [27]:
def glove_vec(glove_file):
    with open (glove_file,'r') as f:
        words = set()
        words_to_vec = {}
        for line in f:
            line = line.strip().split()
            current_word = line[0]
            words.add(current_word)
            words_to_vec[current_word] = np.array(line[1:],dtype=np.float64)
            
        i = 1
        word_to_index = {}
        index_to_word = {}
        for w in sorted(words):
            word_to_index[w] = i
            index_to_word[i] = w
            i = i+1
            
    return word_to_index, index_to_word, words_to_vec

In [28]:
word_to_index, index_to_word, words_to_vec = glove_vec('../input/word2vecembedding/glove.6B.100d.txt')

In [29]:
from keras.models import Model,load_model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
from keras import regularizers

In [34]:
def sentence_to_indices(X,word_to_index,max_len):
    m = X.shape[0]
    X_indices = np.zeros((m,max_len))
    for i in range(m):
        j = 0
        sentence_word = [word for word in X[i][0].split()]
        #sentence_word = [word.lower() for word in X[i][0].split()]
        for w in sentence_word:
            if w in word_to_index.keys():
                X_indices[i, j] = word_to_index[w]
            else:
                X_indices[i, j] = word_to_index['unk']
            j =j + 1
            if j == maxLen:
                break
            
    return X_indices

In [31]:
def pretrained_embedding_layer(words_to_vec,word_to_index):
    vocab_length = len(word_to_index) + 1
    emb_dims = words_to_vec['apple'].shape[0]
    emb_matrix = np.zeros((vocab_length,emb_dims))
    
    for word,index in word_to_index.items():
        emb_matrix[index, :] = words_to_vec[word]
        
    embedding_layer = Embedding(vocab_length, emb_dims, trainable=False)
    
    embedding_layer.build((None,))
    
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [23]:
def label_to_emoji(b):
    return emoji.emojize(emoji_dictionary[str(b)], use_aliases=True)

In [24]:
emoji_dictionary = {"0": ":sob:",    
                    "1": ":pensive:",
                    "2": ":smile:",
                    "3": ":smiley:"}

In [25]:
def user(c):
    c = np.asarray([[c]])
    corpus = []
    frame = re.sub('[^a-zA-Z]', ' ', c[0][0])
    frame = frame.lower()
    corpus.append(frame)
    c_new = np.asarray(corpus)
    c_new = np.reshape(c_new, (len(corpus),1))
    indices = sentence_to_indices(c_new, word_to_index, max_len=40)
    a = model.predict(indices)

    if a<=0.3:
        b=0
        print("Highly Negative: ", label_to_emoji(b))
    elif a<=0.5:
        b=1
        print("Negative: ", label_to_emoji(b))
    elif a>=0.8:
        b=2
        print("Highly positive: ", label_to_emoji(b))
    else:
        b=3
        print("Positive: ", label_to_emoji(b))
    

In [ ]:
def sentiment(input_shape,word_to_index,words_to_vec):
    sentence_indices = Input(input_shape, dtype = 'int32')
    
    embedding_layer = pretrained_embedding_layer(words_to_vec,word_to_index)
    
    embedding = embedding_layer(sentence_indices)
    
    
    #layer1
    X = LSTM(128, return_sequences = True)(embedding)
    
    #layer2
    X = LSTM(128, return_sequences = False)(X)
    
    #dense
    X = Dense(1)(X)
    
    #softmax
    X = Activation('sigmoid')(X)
    
    model = Model(inputs = sentence_indices, output = X)
    
    return model

In [ ]:
l = []
for i in range(len(corpus)):
    length = len(corpus[i])
    l.append(length)
l.sort(reverse=True)

In [ ]:
l[0]

In [19]:
model = load_model("../input/final-model1/highbrowsent_fmodel(1).h5")

In [20]:
maxLen = 40

In [21]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 40)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 40, 100)           40000100  
_________________________________________________________________
lstm_1 (LSTM)                (None, 40, 128)           117248    
_________________________________________________________________
dropout_1 (Dropout)          (None, 40, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129 

In [22]:
model.compile(optimizer='adam', loss = 'binary_crossentropy',metrics = ['accuracy'])

In [ ]:
X_train_input = sentence_to_indices(X_train,word_to_index,maxLen)

In [ ]:
X_val_input = sentence_to_indices(X_val,word_to_index,maxLen)

In [ ]:
history = model.fit(X_train_input, Y_train, epochs = 10, batch_size = 512, shuffle=True, validation_data=(X_val_input, Y_val))

In [ ]:
X_test_input = sentence_to_indices(X_test,word_to_index,maxLen)

In [ ]:
loss, acc = model.evaluate(X_test_input, Y_test)

In [ ]:
print(loss,acc)

In [ ]:
model.save("highbrowsent_f_model.h5")

In [ ]:
from IPython.display import FileLink

In [ ]:
FileLink("highbrowsent_f_model.h5")